<a href="https://colab.research.google.com/github/jaisidhsingh/Real-Time-Face-Mask-Detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

 95% 314M/330M [00:03<00:00, 72.4MB/s]
100% 330M/330M [00:03<00:00, 96.5MB/s]


In [ ]:
!unzip face-mask-12k-images-dataset.zip

In [ ]:
import os
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [ ]:
dir_type = ["Train","Test","Validation"]
cat = ["WithMask","WithoutMask"]

training_data = []
test_data = []
val_data=[]

IMG_SIZE = 100

path = "Face Mask Dataset/"

def create_data():
    for item in dir_type:
        data_path = path+item
        X = []
        y = []
        for folder in os.listdir(data_path):
            if cat.index(folder) == 0:
                labels = [1.0, 0.0]
            else:
                labels = [0.0, 1.0]
            for image in os.listdir(data_path+"/"+folder):
                img = cv2.imread(data_path+"/"+folder+"/"+image)
                # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (100, 100))
                X.append(img)
                y.append(labels)
            if item == "Train":
                training_data.append((X, y))
            if item=="Test":
                test_data.append((X, y))
            if item=="Validation":
                val_data.append((X, y))

create_data()

In [ ]:
train_images = np.array(training_data[0][0])/255.0
train_images = train_images.reshape((10000, 100, 100,3))
train_labels = np.array(training_data[0][1])

test_images = np.array(test_data[0][0])/255.0
test_images = test_images.reshape(-1, 100, 100, 3)
test_labels = np.array(test_data[0][1])

val_images = np.array(val_data[0][0])/255.0
val_images = val_images.reshape(-1, 100, 100, 3)
val_labels = np.array(val_data[0][1])
print(train_images.shape)
print(test_images.shape)
print(val_images.shape)

(10000, 100, 100, 3)
(992, 100, 100, 3)
(800, 100, 100, 3)


In [ ]:
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Dense, Flatten
from tensorflow.keras.applications import VGG19


data_augmentation = keras.Sequential(
    [
        keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        keras.layers.experimental.preprocessing.RandomRotation(0.1),
        keras.layers.experimental.preprocessing.RandomZoom((0.2, 0.3)),
    ]
)


base_model = tf.keras.applications.VGG19(input_shape=(100, 100, 3), include_top=False, weights='imagenet')
for layer in base_model.layers[2:]:
    layer.trainable = False
# base_model.trainable =False
flattening = tf.keras.layers.Flatten()
pre_layer1 = tf.keras.layers.Dense(256,activation="relu")
pre_layer2 = tf.keras.layers.Dense(64, activation="relu")
prediction_layer = tf.keras.layers.Dense(2, activation="softmax")

inputs = tf.keras.Input(shape=(100, 100, 3))
x = data_augmentation(inputs)
x = base_model(x, training=True)
# x = tf.keras.layers.Dropout(0.2)(x)
x = flattening(x)
x = pre_layer1(x)
x=pre_layer2(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 100, 100, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 3, 3, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               1179904   
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 130 

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

epochs = 10
model.fit(train_images, train_labels, batch_size=32, epochs=epochs, validation_data=(val_images, val_labels), shuffle=True)

Epoch 1/10
313/313 [==============================] - 61s 88ms/step - loss: 0.0930 - accuracy: 0.9649 - val_loss: 0.0103 - val_accuracy: 0.9962
Epoch 2/10
313/313 [==============================] - 27s 85ms/step - loss: 0.0384 - accuracy: 0.9857 - val_loss: 0.0057 - val_accuracy: 0.9987
Epoch 3/10
313/313 [==============================] - 27s 87ms/step - loss: 0.0307 - accuracy: 0.9891 - val_loss: 0.0097 - val_accuracy: 0.9987
Epoch 4/10
313/313 [==============================] - 28s 90ms/step - loss: 0.0278 - accuracy: 0.9897 - val_loss: 0.0169 - val_accuracy: 0.9937
Epoch 5/10
313/313 [==============================] - 28s 88ms/step - loss: 0.0253 - accuracy: 0.9925 - val_loss: 0.0099 - val_accuracy: 0.9987
Epoch 6/10
313/313 [==============================] - 27s 87ms/step - loss: 0.0220 - accuracy: 0.9922 - val_loss: 0.0121 - val_accuracy: 0.9987
Epoch 7/10
313/313 [==============================] - 28s 89ms/step - loss: 0.0214 - accuracy: 0.9936 - val_loss: 0.0193 - val_accuracy:

In [ ]:
# score the model
score = model.evaluate(test_images, test_labels)
print("loss: ", score[0])
print("accuracy : ", score[1]*100, "%")

31/31 [==============================] - 1s 42ms/step - loss: 0.0138 - accuracy: 0.9960
loss:  0.013826403766870499
accuracy :  99.59677457809448 %


In [ ]:
model.save("mask_detector.h5")

In [ ]:
model.save("./")

INFO:tensorflow:Assets written to: ./assets
